In [1]:
import pandas as pd
import folium

In [2]:
path = './df_fin.csv'
df = pd.read_csv(path)
df

Unnamed: 0          관광지명                  도로명주소 중분류 카테고리  소분류 카테고리  \
0              0         금곡도서관    부산 북구 효열로203번길 34-0     문화시설  문화생활시설기타   
1              1           수덕사      부산 북구 상학산복길 107-0    역사관광지         절   
2              2  구포대리마을커뮤니티센터  부산 북구 백양대로1172번길 18-0     문화시설  문화생활시설기타   
3              3           만덕사       부산 북구 구만덕로 63-13    역사관광지         절   
4              4           만덕사   부산 북구 덕천로386번길 47-18    역사관광지         절   
...          ...           ...                    ...      ...       ...   
1377        1377           연화사      부산 연제구 경기장로 25-25    역사관광지         절   
1378        1378           월륜사       부산 연제구 고분로 284-0    역사관광지         절   
1379        1379       밤골작은도서관    부산 연제구 금련로24번길 33-0     문화시설  문화생활시설기타   
1380        1380           공덕사      부산 연제구 황령산로 497-1    역사관광지         절   
1381        1381          솔갤러리      부산 연제구 거제대로 180-0     문화시설    화랑/전시관   

             gu                     주소         위도          경도  cnt  
0        Buk-gu    부산 북구 효열로203번길 34-0  35.263010  129.017510   65  
1        Buk-gu      부산 북구 상학산복길 107-0  35.216131  129.039053   65  
2        Buk-gu  부산 북구 백양대로1172번길 18-0  35.206401  129.005594   65  
3        Buk-gu       부산 북구 구만덕로 63-13  35.214870  129.041720   65  
4        Buk-gu   부산 북구 덕천로386번길 47-18  35.208188  129.044809   65  
...         ...                    ...        ...         ...  ...  
1377  Yeonje-gu                 부산 연제구  35.176500  129.079300   55  
1378  Yeonje-gu         부산 연제구 고분로 284  35.185649  129.087249   55  
1379  Yeonje-gu    부산 연제구 금련로24번길 33-0  35.171950  129.095164   55  
1380  Yeonje-gu      부산 연제구 황령산로 497-1  35.158915  129.082773   55  
1381  Yeonje-gu        부산 연제구 거제대로 180  35.184291  129.071174   55  

[1382 rows x 10 columns]

In [3]:
# 지도 데이터 로드
# 부산 구군 행정구역이 나눠져 있음
file = open('./map4.geojson', 'r', encoding='utf-8')
map = file.read()

In [4]:
# 로드한 지도 데이터를 투사할 기본 지도 설정
m = folium.Map(
    location = [35.210419382538085, 129.08972725789795]
    , zoom_start = 11
)
m

In [5]:
# 지도에 색을 입힐 때 세세하게 하기 위해 구간 분할
# quantile() 안에는 0~1 사이 값이 들어감
myscale = (df['cnt'].quantile((0,0.2,0.3,0.4,0.6,0.7,0.8,0.9,0.98,1))).tolist()

In [6]:
# 지도 그려 투사
choropleth = folium.Choropleth(
    geo_data = map, # 사용할 지도 데이터
    data = df, # 분석 대상 데이터 프레임
    columns = ('gu', 'cnt'), # 지도에 사용할 컬럼 지정
    key_on = "feature.properties.gu", # 색을 다르게 할 기준 데이터 지정
    fill_color = 'YlOrRd', # 색 설정
    fill_opacity = 9, # 투명도 설정
    threshold_scale = myscale # 색 입힐 구간 설정 / 구간을 세분화할수록 색 단계 많아짐
).add_to(m) # 지도에 투사하라

# 마우스를 갖다대면 정보가 나오게 하는 기능
choropleth.geojson.add_child(
    folium.features.GeoJsonTooltip(['name','count', 'pop'], aliases = ['구군명', '문화시설수', '인구'], labels= True)
)

# 지도 확인
m